In [1]:
import os
import pandas as pd

import requests
from bs4 import BeautifulSoup

import re
import json

## Search page

In [1]:
url = "https://www.wework.com/search"
page = BeautifulSoup(requests.get(url).content,'lxml')   


In [2]:
pattern = re.compile(r"window.clientSideInjectionVars = (.+);")
script = page.find('script',text=pattern)
info = pattern.search(script.prettify()).group(1)
data = json.loads(info)
data['buildings'][:2]

[{'1': '32a3b948-6dde-4a46-9620-e6b9c912d110',
  '2': '1001-woodward-ave--detroit--MI',
  '3': '1001 Woodward Ave',
  '5': '//cdn.wework.com/locations/image/70aa70a2-1fa8-11ea-b072-0a5bc5747799/Web_72DPI-20191105_1001_Woodward_Ave_Detroit_015.jpg',
  '8': '42.3320134',
  '9': '-83.0476329',
  '10': 238,
  '11': 'Detroit',
  '17': '2021-12-09',
  '20': '2021-12-09',
  '23': '2022-02-01',
  '26': '2022-02-01',
  '29': '2022-02-01',
  '32': '2022-02-01',
  '38': '2022-02-01',
  '200': ['bikeStorage',
   'wellnessRoom',
   'recordingStudio',
   'recreationalGames'],
  '203': '53ac3eb8-841a-11e6-a3b7-0a488af3e541',
  '204': 1,
  '205': 1,
  '206': ['46b2785e-ff00-4401-82c7-31009328448b',
   'ca04dfa5-b8f4-4c80-b36e-f9547a23f654'],
  '207': 'US',
  '208': ['46b2785e-ff00-4401-82c7-31009328448b',
   'ca04dfa5-b8f4-4c80-b36e-f9547a23f654'],
  '209': True},
 {'1': '5a1a7d25-db1e-4d78-9d21-0034d0637ca5',
  '2': '100-harris-st--sydney--NSW',
  '3': '100 Harris St',
  '5': '//cdn.wework.com/locati

In [24]:
len(data['buildings'])

738

In [14]:
img_link_list = []
for b in data['buildings']:
    img_link_list.append(b['5'])

In [23]:
img_directory = 'image'
wework_directory = 'image/wework'

if not os.path.exists(img_directory):
    os.mkdir(img_directory)
if not os.path.exists(wework_directory):
    os.mkdir(wework_directory)

for i,link in enumerate(img_link_list):
    img = requests.get('http:'+link).content
    with open(wework_directory+'/%d.jpg'%i,'wb') as f:
        f.write(img)

In [3]:
buildings_data = pd.DataFrame(data['buildings'])
buildings_data.to_csv('wework_buildings_data.csv',index=False)

In [4]:
with open('wework_buildings_data.json','w') as f:
    json.dump(data['buildings'],f)

## Building view page

In [2]:
with open('wework_buildings_data.json','r') as f:
    buildings_data = json.load(f)

In [3]:
building_links = []
for b in buildings_data:
    building_links.append('https://www.wework.com/buildings/'+b['2'])

In [8]:
wework_directory = 'image/wework'
images_data = []
for i,link in zip(range(270,len(building_links)),building_links[270:]):
    building_dir = wework_directory+'/%d'%i
    if not os.path.exists(building_dir):
        os.mkdir(building_dir)
    
    b_page = BeautifulSoup(requests.get(link).content,'lxml')
    try:
        slider = b_page.find('div',class_='building-slider-container')
        images = slider.find_all('li')

        if i%5==0:
            print('building %d with %d images'%(i,len(images)))
            with open('wework_images_data.json','w') as f:
                json.dump(images_data,f)

        for j,img in enumerate(images):
            img_data = {}
            img_data['building_id'] = 1
            img_data['image_id'] = j
            img_data['image_link'] = img.find('img').get('src') or img.find('img').get('data-src')

            try:
                img_data['image_label'] = img.find('div',class_='slider__caption').get_text().strip()
            except:
                pass
            images_data.append(img_data)

            img = requests.get(img_data['image_link']).content
            with open(building_dir+'/%d.jpg'%j,'wb') as f:
                f.write(img)
    except:
        print('Error when scrapping building %d'%i)
        continue

building 270 with 40 images
Error when scrapping building 273
building 275 with 32 images
building 280 with 26 images
Error when scrapping building 281
building 285 with 28 images
building 290 with 14 images
building 295 with 28 images
Error when scrapping building 296
Error when scrapping building 297
building 300 with 8 images
Error when scrapping building 302
building 305 with 18 images
building 310 with 16 images
building 315 with 34 images
building 320 with 10 images
Error when scrapping building 324
building 325 with 8 images
Error when scrapping building 328
building 330 with 24 images
building 335 with 12 images
Error when scrapping building 336
building 340 with 16 images
building 345 with 12 images
building 350 with 14 images
Error when scrapping building 352
building 355 with 20 images
building 360 with 12 images
Error when scrapping building 365
Error when scrapping building 367
Error when scrapping building 368
Error when scrapping building 369
Error when scrapping buildin

In [22]:
wework_directory = 'image/wework'
images_data_0 = []
for i,link in enumerate(building_links):
    building_dir = wework_directory+'/%d'%i
    if not os.path.exists(building_dir):
        os.mkdir(building_dir)
    
    b_page = BeautifulSoup(requests.get(link).content,'lxml')
    try:
        slider = b_page.find('div',class_='building-slider-container')
        images = slider.find_all('li')

        if i%5==0:
            print('building %d with %d images'%(i,len(images)))
            with open('wework_images_data_all.json','w') as f:
                json.dump(images_data_0,f)

        for j,img in enumerate(images):
            img_data = {}
            img_data['building_id'] = i
            img_data['image_id'] = j
            img_data['image_link'] = img.find('img').get('src') or img.find('img').get('data-src')

            try:
                img_data['image_label'] = img.find('div',class_='slider__caption').get_text().strip()
            except:
                pass
            images_data_0.append(img_data)

#             img = requests.get(img_data['image_link']).content
#             with open(building_dir+'/%d.jpg'%j,'wb') as f:
#                 f.write(img)
    except:
        print('Error when scrapping building %d'%i)
        continue

building 0 with 20 images
building 5 with 22 images
building 10 with 16 images
building 15 with 12 images
building 20 with 12 images
building 25 with 18 images
Error when scrapping building 26
building 30 with 8 images
Error when scrapping building 34
building 35 with 14 images
Error when scrapping building 38
building 40 with 40 images
building 45 with 40 images
Error when scrapping building 48
building 50 with 12 images
building 55 with 12 images
building 60 with 6 images
Error when scrapping building 61
Error when scrapping building 62
building 65 with 12 images
building 70 with 14 images
building 75 with 28 images
building 80 with 16 images
building 85 with 26 images
Error when scrapping building 89
Error when scrapping building 90
building 95 with 38 images
building 100 with 12 images
building 105 with 18 images
building 110 with 24 images
building 115 with 12 images
building 120 with 28 images
building 125 with 16 images
building 130 with 12 images
building 135 with 24 images
bui

In [9]:
buildings_data[736]

{'1': 'a62166df-0e69-4ad6-b66e-4c3055df94b6',
 '2': 'yuexiu-financial-building--guangzhou',
 '3': 'Yuexiu Financial Building',
 '5': '//cdn.wework.com/locations/image/0c6aa44c-2f0b-11e9-b48f-1202be33576a/webimage-7B83D4BA-7466-4838-970AEDAAAF4E2A2D.jpg',
 '8': '23.12397798',
 '9': '113.3261322',
 '10': 268,
 '11': 'Guangzhou',
 '203': '1722c1d6-9f1d-11e8-ab41-1202be33576a',
 '204': 1,
 '206': [],
 '207': 'CN',
 '208': [],
 '209': None}

In [13]:
buildings_data[89]

{'1': '1f0a2462-f9f8-49bb-99b4-fbc5381675df',
 '2': '188-huaihai-middle-lu--shanghai--31',
 '3': '188 Huaihai Middle Lu',
 '5': '//cdn.wework.com/locations/image/eed4fed6-d0b4-11e9-a33c-0a80a47aa582/Web_150DPI-20190115_WeWork_363_Changping_Lu_-_Common_Areas_-_Wide-2.jpg',
 '8': '31.225034',
 '9': '121.476904',
 '10': 996,
 '11': 'Shanghai',
 '200': ['mothersRoom',
  'espressoBar',
  'naturalLight',
  'outdoorSpace',
  'recreationalGames',
  'eventSpace'],
 '203': 'bcccbdcd-9a23-4586-81cd-378d9e2cbae5',
 '204': 1,
 '206': [],
 '207': 'CN',
 '208': [],
 '209': None}

In [26]:
with open('wework_images_data_all.json','w') as f:
    json.dump(images_data_0,f)

In [27]:
with open('wework_images_data_all.json','r') as f:
    images_data_ = json.load(f)

In [29]:
len(pd.DataFrame(images_data_)['building_id'].unique())

652

In [30]:
len(images_data_)

11732

In [34]:
pd.DataFrame(images_data_)['image_label'].value_counts()

Common Area                       2305
Conference Room                    600
Kitchen                            556
Private Office                     390
Community Bar                      302
                                  ... 
Rendering show: Hot Desk Area        1
Unique Interior design               1
Rendering show: Private Office       1
Reception                            1
Rendering Shown: Outdoor Space       1
Name: image_label, Length: 158, dtype: int64

In [35]:
pd.DataFrame(images_data_).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11732 entries, 0 to 11731
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   building_id  11732 non-null  int64 
 1   image_id     11732 non-null  int64 
 2   image_link   11732 non-null  object
 3   image_label  5857 non-null   object
dtypes: int64(2), object(2)
memory usage: 366.8+ KB
